In [ ]:
!pip install transformers
!pip install accelerate
!pip install einops
!pip install bitsandbytes
!pip install langchain==0.0.220 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "MBZUAI/LaMini-T5-738M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map = "auto", torch_dtype = torch.float32)

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipeline = pipeline(
  "text2text-generation",
  model = model,
  tokenizer = tokenizer,
  max_length = 256,
  do_sample = True,
  temperature = 0.3,
  top_p = 0.95
)

llm = HuggingFacePipeline(pipeline = pipeline)

In [ ]:
prompt = "Write me an article about SageMaker"

In [ ]:
generated_text = llm(prompt)
generated_text

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
	# 'SM_NUM_GPUS': json.dumps(1)
    'HF_TASK':'text2text-generation',
    'device_map':'auto',
    'torch_dtype':'torch_float32'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.4.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Write a short article on GenaI",
})

In [ ]:
prompt = "Write a short artcile on GenAI"

#hyperparameter_payload
payload = {
    "inputs":prompt,
    "parameters": {
        "do_sample":True,
        "top_p":0.7,
        "temperature":0.3,
        "top_k":50,
        "max_new_tokens":512,
        "repetition_penalty":1.03
    }
}

#send request to the endpoint
response = predictor.predict(payload)

print(response)

In [ ]:
ENDPOINT = "huggingface-pytorch-tgi-inference-2024-03-01"

In [ ]:
runtime = boto3.client('runtime.sagemaker')

In [ ]:
response = runtime.invoke_endpoint(EndpointName=ENDPOINT, ContentType="application/json", Body=json.dumps(payload))
print(response)

In [ ]:
prediction = json.loads(response['Body'].read().decode('UTF-8'))

In [ ]:
prediction

In [ ]:
prediction[0]['generated_text']